# Env

In [3]:
# %pip install pyvisa

# Check connection to the instrument

In [8]:
import pyvisa

In [9]:
rm = pyvisa.ResourceManager('C:/windows/System32/visa64.dll')
print(rm.list_resources())

('USB0::0x05E6::0x2602::4522205::INSTR',)


In [10]:
smu = rm.open_resource('USB0::0x05E6::0x2602::4522205::INSTR')

In [11]:
print(smu.query("*IDN?"))

Keithley Instruments Inc., Model 2602B, 4522205, 3.3.5



In [37]:
smu.write('*IDN?')
print(smu.read())

Keithley Instruments Inc., Model 2602B, 4522205, 3.3.5



# Measurement

## turn on/off a channel

In [11]:
smu.write('smua.source.output = smua.OUTPUT_ON')

37

In [12]:
smu.write('smua.source.output = smua.OUTPUT_OFF')

38

## measure a resistance (2-wire)

- Physical setup:

  - Keithley channel A - R - Keithley GND 

  - value of R = 9.8e+3 Ohm
  
- **REFERENCE**

  - [1] [Series 2600B manual](C:\Users\20245580\LabCode\Automate_Lab_Instrument\2600BS-901-01F_2600B_Reference_Aug2021.pdf) - section **Ohms programming example**

  - [2] [write? read? or query?](https://www.youtube.com/watch?v=XhUGKqORBGM)

    - [pyvis doc](https://pyvisa.readthedocs.io/en/latest/introduction/communication.html) - section **An example**

   


In [57]:
# Restore 2600B defaults.
smu.write('smua.reset()')
# Select the current source function.
smu.write('smua.source.func = smua.OUTPUT_DCAMPS')
# Set the source range to 10 mA. = resolution
smu.write('smua.source.rangei = 10e-3')
# Set the current source to 10 mA.
smu.write('smua.source.leveli = 10e-3')
# Set the voltage limit to 10 V. = safety
smu.write('smua.source.limitv = 10')
# Enable 2-wire ohms.
smu.write('smua.sense = smua.SENSE_LOCAL')
# Set the voltage range to auto.
smu.write('smua.measure.autorangev = smua.AUTORANGE_ON')
# Turn on output.
smu.write('smua.source.output = smua.OUTPUT_ON')

37

In [74]:
# Retrieve a resistance reading.
smu.query('print(smua.measure.r())')

'5.03968e+00\n'

In [ ]:
# Retrieve a resistance reading.
# ERROR: ValueError: ambiguous ending in termination characters
# smu.read('print(smua.measure.r())')

# another way:
# ERROR: VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.
# print(smu.read())

In [65]:
# Turn off output.
smu.write('smua.source.output = smua.OUTPUT_OFF')

38

## source V, measure I

- Physical setup:

  - Keithley channel A - R - Keithley GND 

  - value of R = 9.8e+3 Ohm

In [75]:
# Restore 2600B defaults.
smu.write('smua.reset()')
# Select voltage source function.
smu.write('smua.source.func = smua.OUTPUT_DCVOLTS')
# Set source range to autorange.
smu.write('smua.source.autorangev = smua.AUTORANGE_ON')
# Set voltage source to 5 V.
smu.write('smua.source.levelv = 5')
# Set current limit to 10 mA.
smu.write('smua.source.limiti = 10e-3')
# Set current range to 10 mA.
smu.write('smua.measure.rangei = 10e-3')
# Turn on output.
smu.write('smua.source.output = smua.OUTPUT_ON')

# # Print and place the current reading in the reading buffer.
# smu.write('smua.reset()')
# print(smua.measure.i(smua.nvbuffer1))

37

In [88]:
# retrieve measured current
smu.query('print(smua.measure.i())')

'-1.09076e-06\n'

In [84]:
# Turn off output.
# smu.write('smua.source.output = smua.OUTPUT_OFF')
smu.write('smua.source.output = smua.OUTPUT_OFF')

38

## source V, get I from internal buffer

- Physical setup:

  - Keithley channel A - R - Keithley GND 

  - value of R = 9.8e+3 Ohm

### M1

In [100]:
# VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.
# smu.query('print(SweepILinMeasureV.source)')

In [ ]:
# VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.
# scripts_ptr = smu.read('script.factory.catalog()')
# print(scripts_ptr)

In [ ]:
# page 3-8
# ======
# Configure smub as source v, measure i
# ======
# Restore 2600B defaults.
smu.write('smua.reset()')

# Select channel A display.
smu.write('display.screen = display.SMUA')

# Display current.
smu.write('display.smua.measure.func = display.MEASURE_DCAMPS')

# Select measure I autorange.
smu.write('smua.measure.autorangei = smua.AUTORANGE_ON')
# Select ASCII data format.
# smu.write('format.data = format.ASCII')

# Clear buffer 1.
smu.write('smua.nvbuffer1.clear()')

# # DOES this matter if the read-in action is defined by 
# # the command `smu.write('smua.measure.i(smua.nvbuffer1)')` (in the for-loop, below)
# # set number of power line cycle (line frequency) for inter ADC integration time
# smu.write('smua.measure.nplc = 1')

# Enable append buffer mode.
smu.write('smua.nvbuffer1.appendmode = 1')

# Enable source value storage.
smu.write('smua.nvbuffer1.collectsourcevalues = 1')

# Enable timestamp
smu.write('smua.nvbuffer1.collecttimestamps = 1')
# timestamp resolution
smu.write('smua.nvbuffer1.timestampresolution = 0.1')

# Set the count to 1.
smu.write('smua.measure.count = 1')

# Select the source voltage function.
smu.write('smua.source.func = smua.OUTPUT_DCVOLTS')

# Set the bias voltage to 0 V.
smu.write('smua.source.levelv = 0.0')

26

In [26]:
# smu.write('smua.source.levelv = 0.4')

In [27]:
# smu.write('smua.measure.i(smua.nvbuffer1)')

In [28]:
import time
# Turn on the output.
smu.write('smua.source.output = smua.OUTPUT_ON')
for v in range(1, 10):
    value = v * 0.01
    smu.write('smua.source.levelv = {}'.format(value))
    time.sleep(5)
    smu.write('smua.measure.i(smua.nvbuffer1)')

In [29]:
# Turn off the output.
smu.write('smua.source.output = smua.OUTPUT_OFF')

38

In [30]:
# Output readings 1 to 100.
print(smu.query('printbuffer(1, smua.nvbuffer1.n, smua.nvbuffer1.readings)'))

# Output source values 1 to 100.
print(smu.query('printbuffer(1, smua.nvbuffer1.n, smua.nvbuffer1.sourcevalues)'))

# Output timestamp 1 to 100. # [s]
print(smu.query('printbuffer(1, smua.nvbuffer1.n, smua.nvbuffer1.timestamp)'))

1.01694e-06, 2.03760e-06, 3.05984e-06, 4.08178e-06, 5.10288e-06, 6.12427e-06, 7.14574e-06, 8.16701e-06, 9.18853e-06

1.00000e-02, 2.00000e-02, 3.00000e-02, 4.00000e-02, 5.00000e-02, 6.00000e-02, 7.00000e-02, 8.00000e-02, 9.00000e-02





In [23]:
print(smu.query('print(smua.nvbuffer1.n)'))

9.00000e+00



In [24]:
# Output timestamp 1 to 100. # [s]
print(smu.query('printbuffer(1, smua.nvbuffer1.n, smua.nvbuffer1.timestamp)'))

### M2

In [33]:
# page 2-64

# Restore 2600B defaults.
smu.write('smua.reset()')
# Set compliance to 1 V.
smu.write('smua.source.limitv = 1')
# Linear staircase sweep
# 1 uA to 10 uA, 0.1 second delay,
# 10 points.
smu.write('SweepILinMeasureV(smua, 1e-6, 10e-6, 0.1, 10)')

47

In [27]:
# Output readings 1 to 100.
print(smu.query('printbuffer(1, smua.nvbuffer1.n, smua.nvbuffer1.readings)'))

# Output source values 1 to 100.
print(smu.query('printbuffer(1, smua.nvbuffer1.n, smua.nvbuffer1.sourcevalues)'))

# Output timestamp 1 to 100. # [s]
print(smu.query('printbuffer(1, smua.nvbuffer1.n, smua.nvbuffer1.timestamp)'))

In [26]:
# Clear buffer 1.
smu.write('smua.nvbuffer1.clear()')

24

## EXP 3 

- 1. instrument setup: smua source v, measure i; smub voltmeter

- 2. setup: smua sig & sumb sig - R (9.8k) - smu gnd

- 3. expected range in measurement:

  - 1. voltage source from smu a: 0.01 V to 1 V

  - 2. current expected range through the resistor: 1 uA to 10 uA

In [32]:
print(10e+3 * 1e-3)

10.0


In [ ]:
# PAGE 2-14
# ======
# Configure smub as only voltmeter
# ======
smu.write('smub.reset()')

# (step 1) Select the current source function.
smu.write('smub.source.func = smub.OUTPUT_DCAMPS')

# (step 2)
## source side
# Set the bias current to 0 A. (source level)
smu.write('smub.source.leveli = 0.0')
# Set the source range to lowest (resolution): 100 nA
smu.write('smub.source.rangei = 100e-9')
# Set the current limit (safety) to be higer than in the expected measurement
smu.write('smub.source.limiti = 1e-3')

## measure side
# ? When selecting as current source, 
# the instrument channel is automatically thought of as voltage meter ?

# Select measure voltage autorange.
smu.write('smub.measure.autorangev = smub.AUTORANGE_ON')

# Enable 2-wire.
smu.write('smub.sense = smub.SENSE_LOCAL')


# page 3-8
# ======
# Configure smua as source v, measure i
# ======
# Restore 2600B defaults.
smu.write('smua.reset()')

# Select channel A display.
smu.write('display.screen = display.SMUA')

# Display current.
smu.write('display.smua.measure.func = display.MEASURE_DCAMPS')

# Select measure I autorange.
smu.write('smua.measure.autorangei = smua.AUTORANGE_ON')

# Select ASCII data format.
# smu.write('format.data = format.ASCII')

# Clear buffer 1.
smu.write('smua.nvbuffer1.clear()')

# # DOES this matter if the read-in action is defined by 
# # the command `smu.write('smua.measure.i(smua.nvbuffer1)')` (in the for-loop, below)
# # set number of power line cycle (line frequency) for inter ADC integration time
# smu.write('smua.measure.nplc = 1')

# Enable append buffer mode.
smu.write('smua.nvbuffer1.appendmode = 1')

# Enable source value storage.
smu.write('smua.nvbuffer1.collectsourcevalues = 1')

# Enable timestamp
smu.write('smua.nvbuffer1.collecttimestamps = 1')
# timestamp resolution
smu.write('smua.nvbuffer1.timestampresolution = 0.1')

# Set the count to 1.
smu.write('smua.measure.count = 1')

# Select the source voltage function.
smu.write('smua.source.func = smua.OUTPUT_DCVOLTS')

# Set the bias voltage to 0 V.
smu.write('smua.source.levelv = 0.0')

26

In [35]:
import time

# Turn on the voltmeter on.
smu.write('smub.source.output = smub.OUTPUT_ON')

# Turn on the output source on.
smu.write('smua.source.output = smua.OUTPUT_ON')

for v in range(1, 10):
    value = v * 0.01
    smu.write('smua.source.levelv = {}'.format(value))
    time.sleep(1)
    print(f"current measured at A: {smu.query('print(smua.measure.i())')}")
    print(f"voltage measured at A: {smu.query('print(smua.measure.v())')}")
    print(f"voltage measured at B: {smu.query('print(smub.measure.v())')}")
    print(f"{'='*5}")
    time.sleep(1)
    

current measured at A: 1.01563e-06

voltage measured at A: 1.00085e-02

voltage measured at B: 9.99861e-03

=====
current measured at A: 2.03697e-06

voltage measured at A: 2.00072e-02

voltage measured at B: 1.99930e-02

=====
current measured at A: 3.05885e-06

voltage measured at A: 3.00085e-02

voltage measured at B: 2.99970e-02

=====
current measured at A: 4.08077e-06

voltage measured at A: 4.00104e-02

voltage measured at B: 3.99946e-02

=====
current measured at A: 5.10239e-06

voltage measured at A: 5.00126e-02

voltage measured at B: 4.99973e-02

=====
current measured at A: 6.12416e-06

voltage measured at A: 6.00119e-02

voltage measured at B: 5.99939e-02

=====
current measured at A: 7.14536e-06

voltage measured at A: 7.00098e-02

voltage measured at B: 6.99918e-02

=====
current measured at A: 8.16737e-06

voltage measured at A: 8.00100e-02

voltage measured at B: 7.99859e-02

=====
current measured at A: 9.18831e-06

voltage measured at A: 9.00089e-02

voltage measured

In [36]:
# Turn off the output.
smu.write('smua.source.output = smua.OUTPUT_OFF')
smu.write('smub.source.output = smub.OUTPUT_OFF')

38

# get source script content inside the Keithley

In [24]:
# Clear buffer 1.
smu.write('smua.nvbuffer1.clear()')

24

In [25]:
print(smu.query('script.factory.scripts.KISweep.list()'))

loadandrunscript KISweep



In [26]:
try:
    for i in range (0, 1000):
        print(smu.query(''))
except:
    print(f"END OF SOURCE SCRIPT")
    # Clear buffer 1.
    smu.write('smua.nvbuffer1.clear()')

--//////////////////////////////////////////////////////////////////////////// 

-- 

-- KISweep 

-- 

-- The functions in this factory script are provided for backward 

-- compatibility with the Series 2600.  The methods used in this factory 

-- script can be used as examples of how to configure and run sweeps. 

-- 

-- You are free to copy, modify, and use this script in any way. 

-- 

--//////////////////////////////////////////////////////////////////////////// 

 

-- Tables of constants that are based on model number 

local l_constants = 

{ 

    ["2601"] = 

        { 

            min_amps    = 1e-12, 

            min_volts   = 1e-6, 

            max_amps    = 3, 

            max_volts   = 40, 

            volts_fmt   = "+00.0000" 

        }, 

    ["2606"] = 

        { 

            min_amps    = 1e-12, 

            min_volts   = 1e-6, 

            max_amps    = 3, 

            max_volts   = 20, 

            volts_fmt   = "+00.0000" 

        }, 

    ["2611"]

# Keithley2600 python library (official)

In [ ]:
# %pip install keithley2600

# chatGPT
# The TCPIP0 connection on the back of a Keithley 2602B SourceMeter refers to its LAN (Ethernet) interface 
# that supports communication over the TCP/IP protocol. 
# This interface allows you to remotely control the instrument using standard network commands over a TCP/IP network.
# BUT, in my case the LAN-connected Keithley is NOT DETECTED in NI-MAX -> CAN NOT BE USED. 

# To use USB-connected Keithley with python `keithley2600`
# - step 1 install PyUSB (similar to Pyvisa but for USB)
#       ```%pip install PyUSB
# - step 2 install the backend for PyUSB, for my windows 11 - 64 system:
#       go to https://libusb.info -> Downloads -> Latest Windows Binaries
#       extract the downloaded folder -> look for `C:\Users\20245580\Downloads\libusb-1.0.29\MinGW64\dll\libusb-1.0.dll` (because of windows, 64x)
#       move the `libusb-1.0.dll` to `C:/Windows/System32/`
# - step 3: check the code below
#       the result must not have error, if returned error that means we have copy the wrong `libusb-1.0.dll` (maybe mistake from the `C:\Users\20245580\Downloads\libusb-1.0.29\MinGW32`)
import ctypes
ctypes.cdll.LoadLibrary("C:/Windows/System32/libusb-1.0.dll")

In [58]:
# %pip install matplotlib

In [7]:
# %pip install PyUSB

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# k = Keithley2600('USB0::0x05E6::0x2602::4522205::INSTR')
# ValueError: Please install PyUSB to use this resource type.
# No module named 'usb'

# after pip install pyusb
# ValueError: PyUSB does not seem to be properly installed.
# Please refer to PyUSB documentation and 
# install a suitable backend like 
# libusb 0.1, libusb 1.0, libusbx, 
# libusb-win32 or OpenUSB.
# No backend available

In [54]:
import pyvisa
rm = pyvisa.ResourceManager('C:/windows/System32/visa64.dll')  # Forces use of NI-VISA backend
print(rm.list_resources())          # Should show USB or TCPIP instrument

('USB0::0x05E6::0x2602::4522205::INSTR',)


## EXP 1

- 1. setup: smua source v, measure i

- 2. physical setup: smu a sig - R (10k) - smu gnd

In [1]:
import time
from  keithley2600 import Keithley2600, ResultTable

# by default, `Keithley2600` uses `pyvisa-py` for backend. While we do not use `pyvisa-py` as backend but `NI-VISA` instead.
# to know where the backend NI-VISA is installed:
#   in the terminal: run `pyvisa-info`
#   look for: `Backends` - `ivi` - `...\visa64.dll` 
# Using the found backend location as below:
# ( see https://github.com/OE-FET/keithley2600/blob/master/README.md -> section **Backend selection**)
k = Keithley2600('USB0::0x05E6::0x2602::4522205::INSTR', visa_library = 'C:/windows/System32/visa64.dll')



# create ResultTable with two columns
rt = ResultTable(
    column_titles=['Voltage', 'Current'],
    units=['V', 'A'],
    params={'recorded': time.asctime(), 'sweep_type': 'iv'},
)

# # create live plot which updates as data is added
# rt.plot(live=True)


# turn on SMUA
k.smua.source.output = k.smua.OUTPUT_ON 

# measure some currents
for v in range(0, 10):
    # It seems that the voltage is not ON?
    v_applied = v*0.01
    k.apply_voltage(k.smua, v_applied)
    time.sleep(1)
    i = k.smua.measure.i()
    rt.append_row([v, i])

k.smua.source.output = k.smua.OUTPUT_OFF   # turn off SMUA

# save the data
rt.save('iv_curve.txt')

## EXP 2

- Repeat of the section **EXP 3** above, but translate the program from pyvisa to keithley 2600B module

In [2]:
import time
from  keithley2600 import Keithley2600, ResultTable
# init the instrument handle
k = Keithley2600('USB0::0x05E6::0x2602::4522205::INSTR', visa_library = 'C:/windows/System32/visa64.dll')

In [5]:
# PAGE 2-14
# ======
# Configure smub as only voltmeter
# ======

# Clear buffer 1.
k.smub.nvbuffer1.clear()

# reset the channel
k.smub.reset()

# (step 1) Select the current source function.
k.smub.source.func = k.smub.OUTPUT_DCAMPS

# (step 2)
## source side
# Set the bias current to 0 A. (source level)
k.smub.source.leveli = 0.0
# Set the source range to lowest (resolution): 100 nA
k.smub.source.rangei = 100e-9
# Set the current limit (safety) to MUST BE higer than in the expected measurement
k.smub.source.limiti = 1e-3

## measure side
# ? When selecting as current source, 
# the instrument channel is automatically thought of as voltage meter ?

# Select measure voltage autorange.
k.smub.measure.autorangev = k.smub.AUTORANGE_ON

# Enable 2-wire.
k.smub.sense = k.smub.SENSE_LOCAL

# page 3-8
# ======
# Configure smua as source v, measure i
# ======

# Restore 2600B defaults.
k.smua.reset()

# Select channel A display.
k.display.screen = k.display.SMUA

# Display current.
k.display.smua.measure.func = k.display.MEASURE_DCAMPS

# Select measure I autorange.
k.smua.measure.autorangei = k.smua.AUTORANGE_ON

# Select ASCII data format.
# smu.write('format.data = format.ASCII')

# Clear buffer 1.
k.smua.nvbuffer1.clear()

# Select the source voltage function.
k.smua.source.func = k.smua.OUTPUT_DCVOLTS

# Set the bias voltage to 0 V.
k.smua.source.levelv = 0.0


In [6]:
import time

# Turn on the voltmeter on.
k.smub.source.output = k.smub.OUTPUT_ON

# Turn on the output source on.
k.smua.source.output = k.smua.OUTPUT_ON

for v in range(1, 10):
    value = v * 0.01
    k.smua.source.levelv = value
    time.sleep(1)

    i_a = k.smua.measure.i()
    print(f"current measured at A: {i_a}")
    
    v_a = k.smua.measure.v()
    print(f"voltage measured at A: {v_a}")

    v_b = k.smub.measure.v()
    print(f"voltage measured at B: {v_b}")

    print(f"{'='*5}")
    time.sleep(1)
    

current measured at A: 1.01575e-06
voltage measured at A: 0.0100085
voltage measured at B: 0.00999308
=====
current measured at A: 2.03753e-06
voltage measured at A: 0.0200104
voltage measured at B: 0.0199899
=====
current measured at A: 3.05903e-06
voltage measured at A: 0.0300076
voltage measured at B: 0.0299936
=====
current measured at A: 4.08089e-06
voltage measured at A: 0.0400105
voltage measured at B: 0.0399924
=====
current measured at A: 5.10218e-06
voltage measured at A: 0.0500096
voltage measured at B: 0.0499937
=====
current measured at A: 6.12409e-06
voltage measured at A: 0.0600103
voltage measured at B: 0.0599895
=====
current measured at A: 7.14597e-06
voltage measured at A: 0.070011
voltage measured at B: 0.0699882
=====
current measured at A: 8.16701e-06
voltage measured at A: 0.0800086
voltage measured at B: 0.0799798
=====
current measured at A: 9.18862e-06
voltage measured at A: 0.0900105
voltage measured at B: 0.0899828
=====


In [7]:
k.smua.source.output = k.smua.OUTPUT_OFF   # turn off SMUA
k.smub.source.output = k.smub.OUTPUT_OFF   # turn off SMUB

# Trial

In [38]:
smu.write('smua.source.leveli = 1')

24

In [39]:
smu.write('local l_s_leveli = smua.source.leveli')

39

In [42]:
smu.read()

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.